# Выбор локации для скважины

Для нефтедобывающей компании нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Необходимо проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

Напишем функцию для просмотра значений датасета

In [3]:
def inform(data):
    print('*******************************************head************************************************')
    print()
    print()
    print(data.head())
    print('________________________________________describe_______________________________________________')
    print(data.describe())
    print('__________________________________________info_________________________________________________')
    data.info()
    print('________________________________________duplicated_____________________________________________')
    print(data.duplicated().sum())
    print('__________________________________________isnull_______________________________________________')
    print(data.isnull().sum())
    print('_________________________________________nunique_______________________________________________')
    print(data.nunique())
    print('___________________________________________corr________________________________________________')
    print(data.corr())
    print()
    print()
    return data

Цикл, принимающий датасет на вход и запускающий функию

In [4]:
sets = [data0, data1, data2]
for s in sets:
    inform(s)

*******************************************head************************************************


      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
________________________________________describe_______________________________________________
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646   

В целом, данные в порядке.
У второго сета выявлена сильная взаимосвязь колонки f2 и product.

## Обучение и проверка модели

In [5]:
model = LinearRegression()

Преобразуем датасеты в фичи и таргеты. У фичей исключаем столбцы 'id', 'product', а у таргетов оставляем столбец 'product'. Сразу же сбалансируем данные, хоть они и выглядят, как сбалансированные.

In [6]:
features0 = data0.drop(['id','product'], axis=1)
target0 = data0['product']
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(features0, target0, test_size=0.25, random_state=12345)
scaler0 = StandardScaler()
scaler0.fit(features_train0)
train_scaled0 = scaler0.transform(features_train0)
valid_scaled0 = scaler0.transform(features_valid0)

In [7]:
model0 = model.fit(train_scaled0, target_train0)
predict0 = model0.predict(valid_scaled0)
rmse0 = mean_squared_error(target_valid0, predict0)**0.5
mean0=predict0.mean()
print(f'Регион 1. Средний запас предсказанного сырья: {mean0}, RMSE модели = {rmse0}')

Регион 1. Средний запас предсказанного сырья: 92.59256778438035, RMSE модели = 37.5794217150813


In [8]:
features1 = data1.drop(['id','product'], axis=1)
target1 = data1['product']
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(features1, target1, test_size=0.25, random_state=12345)
scaler1 = StandardScaler()
scaler1.fit(features_train1)
train_scaled1 = scaler1.transform(features_train1)
valid_scaled1 = scaler1.transform(features_valid1)

In [9]:
model1 = model.fit(train_scaled1, target_train1)
predict1 = model1.predict(valid_scaled1)
rmse1 = mean_squared_error(target_valid1, predict1)**0.5
mean1 = predict1.mean()
print(f'Регион 1. Средний запас предсказанного сырья: {mean1}, RMSE модели = {rmse1}')

Регион 1. Средний запас предсказанного сырья: 68.728546895446, RMSE модели = 0.893099286775617


In [10]:
features2 = data2.drop(['id','product'], axis=1)
target2 = data2['product']
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(features2, target2, test_size=0.25, random_state=12345)
scaler2 = StandardScaler()
scaler2.fit(features_train2)
train_scaled2 = scaler2.transform(features_train2)
valid_scaled2 = scaler2.transform(features_valid2)

In [11]:
model2 = model.fit(train_scaled2, target_train2)
predict2 = model2.predict(valid_scaled2)
rmse2 = mean_squared_error(target_valid2, predict2)**0.5
mean2 = predict2.mean()
print(f'Регион 1. Средний запас предсказанного сырья: {mean2}, RMSE модели = {rmse2}')

Регион 1. Средний запас предсказанного сырья: 94.96504596800489, RMSE модели = 40.02970873393434


Создадим таблицу, отражающую результаты среднего запаса регионов и RMSE моделей

In [12]:
status = pd.DataFrame([['Регион 0', mean0,     rmse0],
                  ['Регион 1',    mean1,   rmse1],
                  ['Регион 2',  mean2, rmse2]],
columns=['Регионы', 'Средний запас предсказанного сырья', 'RMSE модели'])
status.sort_values('RMSE модели').reset_index(drop=True)

,Регионы,Средний запас предсказанного сырья,RMSE модели
0,Регион 1,68.728547,0.893099
1,Регион 0,92.592568,37.579422
2,Регион 2,94.965046,40.029709


На данный момент лучшим регионом показал себя регион 1 с RMSE = 0.893 и средним запасом предсказанного сырья в 68.728547 ед. Такое значение RMSE достигается за счет колонки 'f2' в датасете региона 1.

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных.

In [13]:
all_point = 500 # 500  точек для разработки
best_point = 200 # 200 лучших точек для разработки
budget = 10*10**9 # региональный бюджет
per_barrel = 450 # цена за 1 баррель
per_product = 450*10**3 # цена за единицу продукта (1000 баррелей)
prop_loss = '2.5%' # граница вероятности убытков
best_gerion = 'max_profit' # лучший регион с наибольшей средней прибылью

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [14]:
min_profitable = budget / per_product
min_count_per_point = min_profitable / best_point
print('Достаточный обьем для безубыточной разработки новой скважины равен:',min_count_per_point)

Достаточный обьем для безубыточной разработки новой скважины равен: 111.11111111111111


Сравним полученный объём сырья со средним запасом в каждом регионе

In [15]:
status['Необходимое кол-во сырья'] = min_count_per_point

difference = []

for value in status["Средний запас предсказанного сырья"]:
    if value >= min_count_per_point:
        difference.append(min_count_per_point - value)
    else:
        difference.append(min_count_per_point - value)

status['Разница'] = difference

In [16]:
status

,Регионы,Средний запас предсказанного сырья,RMSE модели,Необходимое кол-во сырья,Разница
0,Регион 0,92.592568,37.579422,111.111111,18.518543
1,Регион 1,68.728547,0.893099,111.111111,42.382564
2,Регион 2,94.965046,40.029709,111.111111,16.146065


Для регионов разница среднего обьема и безубыточного составила от 16.146 до 42.383

## Расчёт прибыли и рисков 

Напишем функцию для расчета прибыли по регионам для 200 лучших скважин в каждом из них

In [17]:
 def revenue(target, predict):
    target = pd.Series(target).reset_index(drop=True)
    predict = pd.Series(predict).reset_index(drop=True).sort_values(ascending=False)
    target_best = target[predict[:best_point].index]
    revenue = target_best.sum() * per_product - budget
    return revenue
    

In [18]:
print('Прибыль по региону 0 составит:', revenue(target_valid0, predict0))
print('Прибыль по региону 1 составит:', revenue(target_valid1, predict1))
print('Прибыль по региону 2 составит:', revenue(target_valid2, predict2))

Прибыль по региону 0 составит: 3320826043.1398506
Прибыль по региону 1 составит: 2415086696.681511
Прибыль по региону 2 составит: 2710349963.5998325


Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли

In [19]:
def boost(target, propabilities):
    
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = propabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample))

    values = pd.Series(values)
    risk = (values < 0).mean()*100
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    
    return  mean, risk, lower, upper

In [20]:
mean_0, risk_0, lower_0, upper_0 = boost(target_valid0.reset_index(drop=True), predict0)
print('Средняя выручка по региону 0 =', mean_0)
print('Риск убытков по региону 0 =', risk_0)
print('Границы 95%-ого доверительного интервала региона 0:', lower_0, '-', upper_0)

Средняя выручка по региону 0 = 396164984.8023711
Риск убытков по региону 0 = 6.9
Границы 95%-ого доверительного интервала региона 0: -111215545.89049526 - 909766941.5534226


In [21]:
mean_1, risk_1, lower_1, upper_1 = boost(target_valid1.reset_index(drop=True), predict1)
print('Средняя выручка по региону 1 =', mean_1)
print('Риск убытков по региону 1 =', risk_1)
print('Границы 95%-ого доверительного интервала региона 1:', lower_1, '-', upper_1)

Средняя выручка по региону 1 = 456045105.7866608
Риск убытков по региону 1 = 1.5
Границы 95%-ого доверительного интервала региона 1: 33820509.39898363 - 852289453.866036


In [22]:
mean_2, risk_2, lower_2, upper_2 = boost(target_valid2.reset_index(drop=True), predict2)
print('Средняя выручка по региону 2 =', mean_2)
print('Риск убытков по региону 2 =', risk_2)
print('Границы 95%-ого доверительного интервала региона 2:', lower_2, '-', upper_2)

Средняя выручка по региону 2 = 404403866.5683568
Риск убытков по региону 2 = 7.6
Границы 95%-ого доверительного интервала региона 2: -163350413.39560106 - 950359574.9237995


In [23]:
best = pd.DataFrame([['Регион 0', mean_0, risk_0, lower_0, upper_0],
                  ['Регион 1',    mean_1, risk_1, lower_1, upper_1],
                  ['Регион 2',    mean_2, risk_2, lower_2, upper_2]],
columns=['Регионы', 'Средняя выручка по региону', 'Риск убытков по региону', 'Нижняя граница доверительного интервала', 'Верхняя граница доверительного интервала'])
best.sort_values('Риск убытков по региону').reset_index(drop=True)

,Регионы,Средняя выручка по региону,Риск убытков по региону,Нижняя граница доверительного интервала,Верхняя граница доверительного интервала
0,Регион 1,4.560451e+08,1.5,3.382051e+07,8.522895e+08
1,Регион 0,3.961650e+08,6.9,-1.112155e+08,9.097669e+08
2,Регион 2,4.044039e+08,7.6,-1.633504e+08,9.503596e+08


Лучшим регионом для разработки скважин с средней выручкой 456045105, риском убытков в 1.5% (по условию менее 2.5%) , и границами доверительного интервала 33820509-852289453, является регион 1.